<a href="https://colab.research.google.com/github/pj0620/google-colab-notebooks/blob/main/Minesweeper_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import random
from scipy.signal import convolve2d
%pip install stable-baselines3[extra]

import numpy as np
import gymnasium as gym
from gymnasium import spaces

!pip install gymnasium[atari]
!pip install gymnasium[accept-rom-license]

!apt-get install swig cmake ffmpeg
!pip install git+https://github.com/DLR-RM/rl-baselines3-zoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 15.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.4 MB/s eta 0:00:00
Using cached Farama_Notifications-0.0.4-py3-none-any.whl (2.5 kB)
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=d371d435ce5edfcbde0bba34849f02ac9c03e62ae8ff83b12930d9a8fd5014ac
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
  Attempting uninstall: shimmy
    Found existing ins

  Attempting uninstall: autorom
    Found existing installation: AutoROM 0.6.1
    Uninstalling AutoROM-0.6.1:
      Successfully uninstalled AutoROM-0.6.1
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig4.0 amd64 4.0.2-1ubuntu1 [1,110 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig all 4.0.2-1ubuntu1 [5,632 B]
Fetched 1,116 kB in 0s (8,358 kB/s)
Selecting previously un

## Hyperparameter configurations





Create a file named `dql.yml` with the following contents

```
Minesweeper-v1:
  frame_stack: 1
  policy: 'CnnPolicy'
  n_timesteps: !!float 1e6
  buffer_size: 100000
  learning_rate: !!float 1e-4
  batch_size: 32
  learning_starts: 100000
  target_update_interval: 1000
  train_freq: 4
  gradient_steps: 1
  exploration_fraction: 0.1
  exploration_final_eps: 0.01
  # If True, you need to deactivate handle_timeout_termination
  # in the replay_buffer_kwargs
  optimize_memory_usage: False
```


## Custom gymnasium env

In [49]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
from skimage.transform import resize

CLICK_BOMB = "click-bomb"
GAME_WIN = "game-win"
GAME_LOSE = "game-lose"
CLICK_VISIBLE = "click-visible"
CLICK_GUESS = "click-guess"
CLICK_VALID = "click-valid"

class MinesweeperEnvironment(gym.Env):
  # Because of google colab, we cannot implement the GUI ('human' render mode)
  metadata = {"render_modes": ["console"]}

  def __init__(self, board_size=9, total_bombs=8, render_mode="console", end_on_bomb=False):
    super().__init__()
    self.render_mode = render_mode
    self.board_size = board_size
    self.total_bombs = total_bombs
    self.end_on_bomb = end_on_bomb

    self.visible = np.zeros((self.board_size, self.board_size), dtype=np.uint8)
    self.set_bombs()
    self.set_values()

    # Define observation space
    # self.observation_space = spaces.Dict({
    #   "visible_vals": spaces.Box(low=0, high=8, shape=(self.board_size, self.board_size), dtype=np.int8),
    #   "visible": spaces.Box(low=0, high=1, shape=(self.board_size, self.board_size), dtype=np.int8)
    # })

    # self.observation_space = spaces.Box(
    #     low=0, high=8, shape=(2 * self.board_size**2,), dtype=np.int8
    # )

    self.observation_space = spaces.Box(
        low=-1, high=1, shape=(self.board_size, self.board_size, 2)
    )

    # Define action space
    self.action_space = spaces.Discrete(self.board_size**2)

    self.last_action = -1

  def set_bombs(self):
    random.seed(10)
    self.bombs = np.zeros(shape=(self.board_size, self.board_size),  dtype=np.uint8)
    placed_bombs = 0
    while placed_bombs < self.total_bombs:
      i = random.randint(0, self.board_size-1)
      j = random.randint(0, self.board_size-1)

      if self.bombs[i][j] == 0:
        self.bombs[i][j] = 1
        placed_bombs += 1

  def set_values(self):
    KERNAL = np.ones((3, 3))
    self.vals = convolve2d(self.bombs, KERNAL, mode='same').astype(np.uint8)

  def reset(self, seed=None, options=None):
    # Reset the environment to an initial state
    self.visible = np.zeros((self.board_size, self.board_size), dtype=np.uint8)

    # same everytime
    # self.set_bombs()
    # self.set_values()
    self.last_action = -1

    return self.get_state(), {}

  def propogate(self, x: int, y: int):
    if self.bombs[x][y] == 1 or self.visible[x][y] == 1:
      return

    self.visible[x][y] = 1
    if self.vals[x][y] == 0:
      for x_k in [x - 1, x, x + 1]:
        for y_k in [y - 1, y, y + 1]:
          if (x_k, y_k) == (x, y):
            continue

          if x_k < 0 or x_k > self.board_size - 1:
            continue

          if y_k < 0 or y_k > self.board_size - 1:
            continue

          self.propogate(x_k, y_k)


  def step(self, action):
    # Implement the logic for taking a step in the environment
    x = action // self.board_size
    y = action % self.board_size

    start_visible_cells = np.sum(self.visible)

    if self.visible[x][y] == 1:
      start_teminated = bool((self.board_size**2 - start_visible_cells) == self.total_bombs)
      return self.get_state(), -0.5, start_teminated, False, {"effect": CLICK_VISIBLE}
      # return self.get_state(), -1, True, True, {}
    elif self.bombs[x][y] == 1:
      if self.end_on_bomb:
        return self.get_state(), -1, True, True, {"effect": CLICK_BOMB}
      else:
        return self.get_state(), -1, False, False, {"effect": CLICK_BOMB}

    next_to_zero = False
    # if start_visible_cells > 1:
    for x_k in [x - 1, x, x + 1]:
      for y_k in [y - 1, y, y + 1]:
        if (x_k, y_k) == (x, y):
          continue

        if x_k < 0 or x_k > self.board_size - 1:
          continue

        if y_k < 0 or y_k > self.board_size - 1:
          continue

        if self.visible[x_k][y_k] == 1:
          next_to_zero = True
          break
    # else:
    #   next_to_zero = True

    self.last_action = action

    if self.vals[x][y] == 0:
      self.propogate(x, y)

    self.visible[x][y] = 1
    end_visible_cells = np.sum(self.visible)
    teminated = bool((self.board_size**2 - end_visible_cells) == self.total_bombs)

    if teminated:
      return self.get_state(), 1, teminated, False, {"effect": GAME_WIN}
    else:
      # print(f"start_visible_cells: {start_visible_cells}")
      # print(f"end_visible_cells: {end_visible_cells}")
      # reward = 2 * (float(end_visible_cells) - float(start_visible_cells)) / (self.board_size**2)
      reward = 0.3 if next_to_zero else -0.3
      info = CLICK_VALID if next_to_zero else CLICK_GUESS
      return self.get_state(), reward, teminated, False, {"effect": info}

  def get_state(self):
    visible_vals = self.visible * self.vals
    visible_vals = (visible_vals.astype(np.float32) - 4) / 4
    visible_scaled = 2 * (self.visible.astype(np.float32) - 0.5)
    return np.stack([visible_vals, visible_scaled], axis=2).astype(np.float32)

  def render(self, mode="console"):
    if self.render_mode != "console":
        raise NotImplementedError("Render mode not supported.")

    # Print the current visible board state
    cells_left = int(self.board_size**2 - np.sum(self.visible))
    print(f"Current Board: {cells_left - self.total_bombs} Cells Left")
    print("# " + " ".join(str(i) for i in range(self.board_size)))
    for i in range(self.board_size):
        print(f"{i} ", end="")
        row = ""
        for j in range(self.board_size):
            if self.visible[i][j] == 1:
                # If the cell is visible, show its value (number of adjacent bombs)
                if self.bombs[i][j] == 1:
                  row += f"B "
                else:
                  row += f"{self.vals[i][j]} "
            else:
                # If the cell is hidden, show an asterisk
                row += "* "
        print(row)
    # print("\n")


from stable_baselines3.common.env_checker import check_env

env = MinesweeperEnvironment()

# Testing env
check_env(env)
print("starting game")
for round in range(1):
  obs, info = env.reset()
  for _ in range(10):
      # Random action
      action = env.action_space.sample()
      obs, reward, terminated, truncated, info = env.step(action)
      if terminated:
          obs, info = env.reset()
      env.render()
      reward_pos = f"{reward * (env.board_size ** 2)} / {env.board_size ** 2}"
      print(f"action: {action} -> {(action // env.board_size, action % env.board_size)}")
      # print(f"reward: {reward if reward <= 0 else reward_pos}")
      print(f"reward: {reward}")
      print("\n")
  env.reset()

starting game
Current Board: 73 Cells Left
# 0 1 2 3 4 5 6 7 8
0 * * * * * * * * * 
1 * * * * * * * * * 
2 * * * * * * * * * 
3 * * * * * * * * * 
4 * * * * * * * * * 
5 * * * * * * * * * 
6 * * * * * * * * * 
7 * * * * * * * * * 
8 * * * * * * * * * 
action: 18 -> (2, 0)
reward: -1


Current Board: 72 Cells Left
# 0 1 2 3 4 5 6 7 8
0 * * * * * * * * * 
1 * * * * * * * * * 
2 * * * * * * * * * 
3 * * * * * * * * * 
4 * * * * * * * * * 
5 * * * * * * * * * 
6 * * * * * * * * * 
7 * * * * * * * * * 
8 1 * * * * * * * * 
action: 72 -> (8, 0)
reward: -0.3


Current Board: 25 Cells Left
# 0 1 2 3 4 5 6 7 8
0 0 0 0 0 0 1 * * * 
1 1 1 0 0 0 1 * * * 
2 * 1 0 0 1 1 * * * 
3 * 1 0 0 1 * * * * 
4 * 1 1 0 1 1 2 1 1 
5 * * 1 0 0 0 0 0 0 
6 * * 1 1 1 1 0 0 0 
7 * * * * * 1 1 1 1 
8 1 * * * * * * * * 
action: 13 -> (1, 4)
reward: -0.3


Current Board: 25 Cells Left
# 0 1 2 3 4 5 6 7 8
0 0 0 0 0 0 1 * * * 
1 1 1 0 0 0 1 * * * 
2 * 1 0 0 1 1 * * * 
3 * 1 0 0 1 * * * * 
4 * 1 1 0 1 1 2 1 1 
5 * * 1 0 0 

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/env_checker.py:54: UserWarning: It seems that your observation  is an image but its `dtype` is (float32) whereas it has to be `np.uint8`. If your observation is not an image, we recommend you to flatten the observation to have only a 1D vector
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/env_checker.py:62: UserWarning: It seems that your observation space  is an image but the upper and lower bounds are not in [0, 255]. Because the CNN policy normalize automatically the observation you may encounter issue if the values are not in that range.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/env_checker.py:75: UserWarning: The minimal resolution for an image is 36x36 for the default `CnnPolicy`. You might need to use a custom features extractor cf. https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html
  warnings.warn(


### Play minesweeper

In [22]:
env = MinesweeperEnvironment(end_on_bomb=True)

terminated = False
obs, info = env.reset()
env.render()
while not terminated:
  actions_str = input("enter row then column with a space(\"X Y\"):").split(" ")
  actions_int = [int(x) for x in actions_str]
  action = actions_int[0] + actions_int[1] * env.board_size
  obs, reward, terminated, truncated, info = env.step(action)
  print(f"reward: {reward}")
  env.render()

Current Board: 73 Cells Left
# 0 1 2 3 4 5 6 7 8
0 * * * * * * * * * 
1 * * * * * * * * * 
2 * * * * * * * * * 
3 * * * * * * * * * 
4 * * * * * * * * * 
5 * * * * * * * * * 
6 * * * * * * * * * 
7 * * * * * * * * * 
8 * * * * * * * * * 
enter row then column with a space("X Y"):0 3
reward: -0.3
Current Board: 72 Cells Left
# 0 1 2 3 4 5 6 7 8
0 * * * * * * * * * 
1 * * * * * * * * * 
2 * * * * * * * * * 
3 1 * * * * * * * * 
4 * * * * * * * * * 
5 * * * * * * * * * 
6 * * * * * * * * * 
7 * * * * * * * * * 
8 * * * * * * * * * 
enter row then column with a space("X Y"):0 4
reward: 0.3
Current Board: 71 Cells Left
# 0 1 2 3 4 5 6 7 8
0 * * * * * * * * * 
1 * * * * * * * * * 
2 * * * * * * * * * 
3 1 * * * * * * * * 
4 1 * * * * * * * * 
5 * * * * * * * * * 
6 * * * * * * * * * 
7 * * * * * * * * * 
8 * * * * * * * * * 
enter row then column with a space("X Y"):5 4
reward: -0.3
Current Board: 70 Cells Left
# 0 1 2 3 4 5 6 7 8
0 * * * * * * * * * 
1 * * * * * * * * * 
2 * * * * * * * * *

## DQN

In [15]:
from math import log

# 18418
log(0.01) / log(0.99975)

18418 / 100_000

0.18418

In [31]:
import torch as th
import torch.nn as nn
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

class CustomCNN(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=256):
        super(CustomCNN, self).__init__(observation_space, features_dim)
        n_input_channels = observation_space.shape[2]  # Should be 2 for (9, 9, 2) input

        # Define a custom CNN architecture
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            # nn.Conv2d(64, 64, kernel_size=3, padding=1),
            # nn.ReLU(),
            # nn.Conv2d(64, 64, kernel_size=3, padding=1),
            # nn.ReLU(),
            nn.Flatten()
        )

        # Calculate the output size of the CNN dynamically
        with th.no_grad():
            sample_input = th.as_tensor(observation_space.sample()[None]).float().permute(0, 3, 1, 2)
            n_flatten = self.cnn(sample_input).shape[1]
            print("Flattened output size after CNN:", n_flatten)  # Debugging statement

        # Define fully connected layers using the computed flattened size
        # self.linear = nn.Sequential(
        #     nn.Linear(n_flatten, 256),  # Use dynamically calculated n_flatten
        #     nn.ReLU(),
        #     nn.Linear(256, 128),
        #     nn.ReLU(),
        #     nn.Linear(256, features_dim)  # Output size of features_dim (256)
        # )
        self.linear = nn.Sequential(
            nn.Linear(n_flatten, features_dim),
            nn.ReLU()
        )

    def forward(self, observations):
        observations = observations.permute(0, 3, 1, 2)  # Rearrange dimensions for Conv2d
        cnn_output = self.cnn(observations)
        return self.linear(cnn_output)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecTransposeImage

vec_env = make_vec_env(MinesweeperEnvironment, n_envs=1)

# vec_env = make_vec_env(MinesweeperEnvironment, n_envs=1)
# vec_env = VecTransposeImage(vec_env)  # Transpose to [batch_size, channels, height, width]

# Optimized hyperparameters for DQN
learning_rate = 1e-4           # Learning rate for weight updates
buffer_size = 500_000            # Replay buffer size
learning_starts = 1000         # Steps before learning begins
batch_size = 64                # Number of samples per training update
train_freq = (4, 'step')    # Frequency of training updates
# train_freq = (1, 'episode')
target_update_interval = 750   # Steps between target network updates
exploration_fraction = 0.25    # Fraction of total timesteps for epsilon decay
exploration_final_eps = 0.01   # Final epsilon value after decay
gamma = 0.99                    # Discount factor for future rewards

#  0.99975 ** x = 0.01 => x = log(0.01) / log(0.99975)

policy_kwargs = dict(
    features_extractor_class=CustomCNN,
    features_extractor_kwargs=dict(features_dim=256)  # Output size of the final layer
)

# Instantiate DQN with improved hyperparameters
model = DQN(
    "CnnPolicy",
    vec_env,
    learning_rate=learning_rate,
    buffer_size=buffer_size,
    learning_starts=learning_starts,
    batch_size=batch_size,
    train_freq=train_freq,
    target_update_interval=target_update_interval,
    exploration_fraction=exploration_fraction,
    exploration_final_eps=exploration_final_eps,
    gamma=gamma,
    policy_kwargs=policy_kwargs,  # Use custom network
    verbose=1
)

# Train the model
model.learn(total_timesteps=500_000)

Using cuda device
Flattened output size after CNN: 2592
Flattened output size after CNN: 2592
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 214      |
|    ep_rew_mean      | -104     |
|    exploration_rate | 0.993    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 7823     |
|    time_elapsed     | 0        |
|    total_timesteps  | 858      |
----------------------------------


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 235      |
|    ep_rew_mean      | -115     |
|    exploration_rate | 0.985    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 1534     |
|    time_elapsed     | 1        |
|    total_timesteps  | 1880     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0271   |
|    n_updates        | 219      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 250      |
|    ep_rew_mean      | -123     |
|    exploration_rate | 0.976    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 1138     |
|    time_elapsed     | 2        |
|    total_timesteps  | 2994     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.00813  |
|    n_updates      

## PPO

In [ ]:
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.env_util import make_vec_env

vec_env = make_vec_env(MinesweeperEnvironment, n_envs=1)

# Define hyperparameters for PPO
learning_rate = 1e-4       # Learning rate for PPO
n_steps = 2048             # Number of steps to run for each environment per update
batch_size = 64            # Batch size for each update
n_epochs = 10              # Number of times to train on each batch
gamma = 0.99               # Discount factor
gae_lambda = 0.95          # GAE lambda, for variance reduction in advantage estimation
clip_range = 0.2           # Clip range for PPO, helps with stable training

# Instantiate PPO with custom hyperparameters
model = PPO(
    "CnnPolicy",           # Policy type, can try "CnnPolicy" for image-based inputs
    vec_env,
    learning_rate=learning_rate,
    n_steps=n_steps,
    batch_size=batch_size,
    n_epochs=n_epochs,
    gamma=gamma,
    gae_lambda=gae_lambda,
    clip_range=clip_range,
    verbose=1               # Verbose output
)

# Train the model
model.learn(total_timesteps=100000)

Using cuda device
Wrapping the env in a VecTransposeImage.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.63e+03 |
|    ep_rew_mean     | -9.6e+05 |
| time/              |          |
|    fps             | 226      |
|    iterations      | 1        |
|    time_elapsed    | 9        |
|    total_timesteps | 2048     |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1.63e+03  |
|    ep_rew_mean          | -9.6e+05  |
| time/                   |           |
|    fps                  | 207       |
|    iterations           | 2         |
|    time_elapsed         | 19        |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 25.478064 |
|    clip_fraction        | 0.66      |
|    clip_range           | 0.2       |
|    entropy_loss         | -2.98     |
|    explained_variance   | -6.08e-06 |
|

## Evaluation

In [58]:
import torch

# Test the trained agent
# using the vecenv
vec_env2 = make_vec_env(MinesweeperEnvironment, n_envs=1, env_kwargs=dict(end_on_bomb=True))
obs = vec_env2.reset()
n_steps = 100
total_substeps = 0
for step in range(n_steps):
    # action, _ = model.predict(obs, deterministic=False)

    # Convert the observation to a tensor and ensure it's on the same device as the model
    obs_tensor = torch.tensor(obs, dtype=torch.float32).to(model.device)

    # Get Q-values directly from the model's q_net
    with torch.no_grad():
        q_values = model.q_net(obs_tensor).cpu().numpy()

    # Flatten the Q-values and get indices in descending order
    ranked_actions = q_values[0].argsort()[::-1]

    print(f"found {len(ranked_actions)} ranked actions")

    for k, action_k in enumerate(ranked_actions):
      total_substeps += 1
      print(f"trying action {k}")
      x = int(action_k) // 9
      y = int(action_k) % 9
      print("Action: ", (x, y), action_k)
      obs, reward, done, info = vec_env2.step([action_k])
      effect = info[0]["effect"]
      print("reward=", reward, "done=", done, "effect=", effect)
      vec_env2.render()

      if effect == GAME_LOSE:
        break

      if reward > 0:
        break

    if done:
        print("Won!" if effect == GAME_WIN else "Lost :(")
        print("total_steps:", step)
        print("total_substeps:", total_substeps)
        break

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


found 81 ranked actions
trying action 0
Action:  (5, 5) 50
reward= [-0.3] done= [False] effect= click-guess
Current Board: 26 Cells Left
# 0 1 2 3 4 5 6 7 8
0 0 0 0 0 0 1 * * * 
1 1 1 0 0 0 1 * * * 
2 * 1 0 0 1 1 * * * 
3 * 1 0 0 1 * * * * 
4 * 1 1 0 1 1 2 1 1 
5 * * 1 0 0 0 0 0 0 
6 * * 1 1 1 1 0 0 0 
7 * * * * * 1 1 1 1 
8 * * * * * * * * * 
trying action 1
Action:  (3, 3) 30
reward= [-0.5] done= [False] effect= click-visible
Current Board: 26 Cells Left
# 0 1 2 3 4 5 6 7 8
0 0 0 0 0 0 1 * * * 
1 1 1 0 0 0 1 * * * 
2 * 1 0 0 1 1 * * * 
3 * 1 0 0 1 * * * * 
4 * 1 1 0 1 1 2 1 1 
5 * * 1 0 0 0 0 0 0 
6 * * 1 1 1 1 0 0 0 
7 * * * * * 1 1 1 1 
8 * * * * * * * * * 
trying action 2
Action:  (4, 3) 39
reward= [-0.5] done= [False] effect= click-visible
Current Board: 26 Cells Left
# 0 1 2 3 4 5 6 7 8
0 0 0 0 0 0 1 * * * 
1 1 1 0 0 0 1 * * * 
2 * 1 0 0 1 1 * * * 
3 * 1 0 0 1 * * * * 
4 * 1 1 0 1 1 2 1 1 
5 * * 1 0 0 0 0 0 0 
6 * * 1 1 1 1 0 0 0 
7 * * * * * 1 1 1 1 
8 * * * * * * * * * 
trying

## Old RL Zoo code

In [ ]:
# from rl_zoo3.train import train
# from gym.envs.registration import register

# register(
#     id='Minesweeper-v1',
#     entry_point='msenv:MinesweeperEnvironment',  # Update '__main__' to the module name if this is not in your main script
#     max_episode_steps=100,  # Adjust based on expected game length
# )

# import gym
# print([k for k in gym.envs.registry.keys() if "Minesweeper" in k])

# !python -m rl_zoo3.train --algo dqn --env Minesweeper-v1 -f logs/ -c dqn.yml
